# ＳＶＭ

In [33]:
#--------インポート--------
import numpy as np
from numpy.random import *
import random
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


In [34]:
#スクラッチでSVMを行うクラス
class ScratchSVMClassifier():

    """
    SVM分類器のスクラッチ実装
    """

    def __init__(self, num_iter, lr, bias, Threshold):#-------------------------
        self.iter = num_iter       #イテレーション回数
        self.lr = lr               #学習率
        self.bias = bias           #バイアス項
        self.Threshold = Threshold #しきい値

    #self.lambda_ラムダの一般式
    def _SVC_hypothesis(self, X, y):#-------------------------

        #yを2次元の配列に変更
        y = y.reshape(-1,1)

        #self.lambda_[i]はy.shape[0]の１００個あるので要素の数１００のラムダのndarrayを作る
        for i in range(y.shape[0]): #y.shape[0] = 100 yのデータ数だけ繰り返す

            #ダイバーの公式のΣ部分の初期値をゼロと定義する
            siguma = 0

            for j in range(len(y)): #len(y) = 100 #★☆★☆ｉとｊの数は同じだと思うが別にしている理由はあるか

                #ダイバーの公式のΣ部分.np.matmul(X[i].T, X[j])はカーネル関数
                siguma += self.lambda_[j] * y[i] * y[j] * np.matmul(X[i].T, X[j])

            #ダイバーの公式の最急降下法の式
            #self.lambda_[i] += self.lr * (1 - siguma)
            self.lambda_[i] = self.lambda_[i] +  self.lr * (1 - siguma)

            #self.lambda_が０以下の場合は０を代入する
            self.lambda_[self.lambda_ <= 0] = 0

        return self.lambda_

    def fit(self, X, y, X_val=None, y_val=None):#-------------------------

        """
        SVM分類器を学習します。
        検証データが入力されている場合は、そのデータに対する精度も反復ごとに計算されます。
        """
        #ランダム番号０
        np.random.seed(0)

        # λの初期値を設定
        self.lambda_ = np.random.rand(X.shape[0], 1) * 0.030 #np.random.randは0以上、1.0未満の(X.shape[0], 1)の配列の乱数　0.030は

        #イテレーション回数繰り返す
        for i in range(self.iter):#イテレーション回数ｉを繰り返す

            #ラムダ一般式を呼び出すメソッド
            self._SVC_hypothesis(X, y)
            # print("イテレーション回数i:{}".format(i)) #------検証中--------
            # print("self.lambda_:{}".format(self.lambda_)) #------検証中--------
            # print("self.lambda_.shape:{}".format(self.lambda_.shape)) #------検証中--------#self.lambda_.shape:(22, 1)
            # print("X.shape:{}".format(X.shape)) #------検証中--------
            # print("y.shape:{}".format(y.shape)) #------検証中--------

        # ------------サポートベクターの抽出、仕様------------

        # self.index_lという空のリストを作り、しきい値より大きいラムダの要素番号をself.index_lに追加していく
        self.index_l = []

        #ラムダの値と要素番号を出力してしきい値より高いものの要素番号をサポートベクターとしてリストに追加
        for j, i in enumerate(self.lambda_):        #enumerateはタプル（インデックス、要素）でｊに要素の番号、ｉに要素を出力する
            if i > self.Threshold:                  #もしラムダの要素がしきい値より大きければ
                self.index_l .append(j)             #大きい閾値の要素番号をself.index_lに格納

        #self.lambda_svは全部のラムダのndarrayからしきい値より大きいラムダのみを抽出したリスト
        self.lambda_sv = self.lambda_[self.index_l]

        #self.X_svは全部のXからラムダがしきい値よりXのみを抽出したリスト（黄色のプロットになる値）サポートベクター
        self.X_sv = X[self.index_l]
        #self.y_svは全部のyからラムダがしきい値よりyのみを抽出したリスト（黄色のプロットになる値）サポートベクター
        self.y_sv = y[self.index_l]

        print("テストデータXのサポートベクターself.X_sv:{}".format(self.X_sv)) #------検証中--------
        print("テストデータｙのサポートベクターself.y_sv:{}".format(self.y_sv)) #------検証中--------


    #ｙの予測値を出力
    def predict(self, X):#-------------------------

        """
        SVM分類器を使ってラベルを推定します。テストデータを引数Xに代入する
        """

        #初期値をゼロとする
        self.y_pred = 0

        #しきい値より大きいラムダのリストのデータ数をｉとし、ダイバーのy_predの一般式にサポートベクターのself.lambda_svとサポートベクターself.X_sv、サポートベクターself.y_svを代入し最適化Ｘ
        for i in range(self.lambda_sv.shape[0]):
             self.y_pred += self.lambda_sv[i] * self.y_sv[i] * np.dot(X, self.X_sv[i].T) #ダイバーのy_predの公式

        #self.y_predが０より大きければ１、そうでなければ－１をself.y_predに代入しself.y_predを完成させる。
        self.y_pred = np.where(self.y_pred > 0, 1, -1)

        return self.y_pred


In [ ]:
#-----------検証用データ-----------

import numpy as np
np.random.seed(seed=0)
n_samples = 30
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X = np.concatenate([f0, f1])
y = np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])

print("検証用データX.shape:{}".format(X.shape)) #------検証中--------
print("検証用データy.shape:{}".format(y.shape)) #------検証中--------
print("検証用データX:{}".format(X)) #------検証中--------
print("検証用データy:{}".format(y)) #------検証中--------

random_index = np.random.permutation(np.arange(n_samples))
X = X[random_index]
y = y[random_index]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [44]:
#--------境界線を描く関数--------

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
def decision_region(X, y, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica']):

    # setting
    scatter_color = ['red', 'blue']
    contourf_color = ['pink', 'skyblue']
    n_class = 2

    # pred
    mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
    mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
    y_pred = model.predict(mesh).reshape(mesh_f0.shape)

    # plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
    plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
    for i, target in enumerate(set(y)):
        plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
    if model==SV:
        plt.scatter(SV.X_sv[:, 0], SV.X_sv[:, 1], s=80, color='yellow', marker='o')
    else:
        plt.scatter(X_train[svc.support_, 0], X_train[svc.support_, 1], s=80, color='yellow', marker='o')
    patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
    plt.legend(handles=patches)
    plt.legend()



In [ ]:
# print("--------Scikit-learnのSVMモジュール--------")
# from sklearn.svm import SVC
# svc = SVC(random_state=0, C=1e10, kernel='linear')
# svc.fit(X_train, y_train)
# svc_pred = svc.predict(X_test)

# #--------境界線を描く関数decision_regionの呼び出し--------
# decision_region(X_train, y_train, svc, step=0.01, title='With Sklearn', xlabel='xlabel', ylabel='ylabel', target_names=["A" ,'B'])


In [47]:
print("--------スクラッチSVMモジュール--------")
SV = ScratchSVMClassifier(num_iter=100, lr=0.01, bias=True ,Threshold=1e-5)
SV.fit(X_train, y_train, X_test, y_test)

#SVインスタンスに、Xテストデータの予測値をSV_predとしてメソッド呼び出し
SV_pred = SV.predict(X_test)
print("SV_pred:{}".format(SV_pred)) #------検証中--------

#--------境界線を描く関数decision_regionの呼び出し--------
decision_region(X, y, SV, step=0.01, title='ScratchLogisticRegression', xlabel='xlabel', ylabel='ylabel', target_names=["A" ,'B'])


--------スクラッチSVMモジュール--------
イテレーション回数i:0
self.lambda_:[[0.00923262]
 [0.01229502]
 [0.00428904]
 [0.01543948]
 [0.00284369]
 [0.00065595]
 [0.00789975]
 [0.0135816 ]
 [0.02246968]
 [0.00692585]
 [0.01172168]
 [0.01322334]
 [0.01088128]
 [0.02403701]
 [0.00147687]
 [0.        ]
 [0.        ]
 [0.01492399]
 [0.02132092]
 [0.02288142]
 [0.02632837]
 [0.02428184]]
self.lambda_.shape:(22, 1)
X.shape:(22, 2)
y.shape:(22,)
イテレーション回数i:1
self.lambda_:[[0.00854176]
 [0.01034246]
 [0.        ]
 [0.01745671]
 [0.00030343]
 [0.        ]
 [0.00847176]
 [0.00702249]
 [0.02112745]
 [0.00607666]
 [0.005512  ]
 [0.01311409]
 [0.00852976]
 [0.02324973]
 [0.00280445]
 [0.        ]
 [0.        ]
 [0.00864691]
 [0.02148176]
 [0.02223772]
 [0.02526854]
 [0.02465728]]
self.lambda_.shape:(22, 1)
X.shape:(22, 2)
y.shape:(22,)
イテレーション回数i:2
self.lambda_:[[0.00958014]
 [0.01029837]
 [0.        ]
 [0.02017567]
 [0.        ]
 [0.        ]
 [0.01087976]
 [0.00248468]
 [0.02137961]
 [0.00636069]
 [0.00109666]
 [0.01